In [ ]:
# MNIST preparation p3 ( working with ImageFolder)
import os
import torch
import torch.utils.data as data
import json
from PIL import Image
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.datasets import ImageFolder


class RavelTransform(nn.Module):
    def forward(self,item):
        return item.ravel()


class DigitNN(nn.Module):
    def __init__(self, input_dim, num_hid, out_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim,num_hid)
        self.layer2 = nn.Linear(num_hid, out_dim)

    def forward(self,x):
        x = self.layer1(x)
        x = nn.functional.relu(x)
        x = self.layer2(x)
        return x

model = DigitNN(28 * 28, 32, 10)

# Setting trnsform
transforms = tfs.Compose([tfs.ToImage(),
                          tfs.Grayscale(), 
                          tfs.ToDtype(torch.float32, scale=True), # converting to float
                          RavelTransform() # or tfs.Lambda(lambda _im: _im.ravel()),
                          ])
d_dataset = ImageFolder("datasets/MNIST/train", transform=transforms)
train_data = data.DataLoader(d_dataset, batch_size=32, shuffle=True)

it = iter(train_data)
x, y  = next(it)



optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

ep = 2
model.train()

for _e in range(ep):
    train_tqdm = tqdm(train_data, leave=True)

    for x_train_img, y_target in train_tqdm:
        predict = model(x_train_img)
        loss = loss_function(predict, y_target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Test
d_test = ImageFolder("datasets/MNIST/test", transform=transforms)
test_data = data.DataLoader(d_test , batch_size=500, shuffle=False)
Q = 0

model.eval()

for x_test, y_test in test_data:
    p = model(x_test) # 500, 10
    p = torch.argmax(p, dim=1) # 500,1
    Q += torch.sum(p == y_test).item()

Q /= len(d_test)
print(Q)



100%|██████████| 1875/1875 [00:18<00:00, 102.27it/s]


0.9365


In [ ]:
# 5
import matplotlib.pyplot as plt

from PIL import Image
# import torchvision.transforms as tfs
import torchvision.transforms.v2 as tfs

img_pil = Image.new(mode="RGB", size=(128, 128), color=(0, 128, 255))
transforms = tfs.ToImage()
img = transforms(img_pil)



In [6]:
# 6

from PIL import Image
# import torchvision.transforms as tfs
import torchvision.transforms.v2 as tfs

img_pil = Image.new(mode="RGB", size=(128, 128), color=(0, 128, 255))
t = tfs.Compose([tfs.Grayscale(), tfs.ToTensor(), tfs.Normalize([0.5],[1.0])])
img = t(img_pil)

c:\Users\pantu\.conda\envs\dl\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [7]:
# 7
from PIL import Image
import torchvision.transforms.v2 as tfs

img_pil = Image.new(mode="RGB", size=(128, 128), color=(0, 128, 255))

# здесь продолжайте программу
t= tfs.Compose([tfs.CenterCrop(64), tfs.Resize(128), tfs.ToTensor()])
img = t(img_pil)


c:\Users\pantu\.conda\envs\dl\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [12]:
# 8
import torch
import torch.nn as nn
# import torchvision.transforms as tfs
import torchvision.transforms.v2 as tfs

# здесь объявляйте класс ToDtypeV1
class ToDtypeV1(nn.Module):
    def __init__(self, dtype, scale):
        super().__init__()
        self.dtype = dtype
        self.scale = scale

    def forward(self,x):
        x = x.to(self.dtype)
        if self.scale and self.dtype in (torch.float16, torch.float32, torch.float64):
            x_min = x.min()
            x_max = x.max()
            x = (x - x_min) / x_max
        return x


H, W = 128, 128
img_orig = torch.randint(0, 256, size=(3, H, W), dtype=torch.uint8) # тензор в программе не менять

img_mean = torch.mean(img_orig.float(), [1,2]) # средние для каждого цветового канала (первая ось)
img_std = torch.std(img_orig.float().flatten(1,2), dim=1) # стандартное отклонение для каждого цветового канала (первая ось)

# здесь продолжайте программу
transforms = tfs.Compose([ToDtypeV1(dtype = torch.float32, scale = False), tfs.Normalize(mean=img_mean, std=img_std)])
img = transforms(img_orig)

In [13]:
# 9
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as tfs
# import torchvision.transforms.v2 as tfs_v2 - недоступен на Stepik

# здесь объявляйте класс AddNoise
class AddNoise(nn.Module):
    def __init__(self, volume):
        super().__init__()
        self.volume = volume
    def forward(self, x):
        noise = torch.randn_like(x) * self.volume
        x = x.to(torch.float32) + noise
        return x
    


img_pil = Image.new(mode="RGB", size=(128, 128), color=(0, 128, 255))

# здесь продолжайте программу
tr = tfs.Compose([tfs.ToTensor(), AddNoise(0.1)])
img = tr(img_pil)

In [2]:
# 10 
import torch

t_rnd = torch.randint(-3, 5, (100, ), dtype=torch.float32) # значения этого тензора в программе не менять
t_mean = torch.mean(t_rnd)
t_max = torch.max(t_rnd[:5])
t_min = torch.max(t_rnd[:-3])
